In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import time
from fuzzywuzzy import process

/opt/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
dff = pd.read_csv("output.csv")

In [4]:
def getCompany(text):
    r = requests.get('https://api.iextrading.com/1.0/ref-data/symbols')
    stockList = r.json()
    return process.extractOne(text, stockList)[0]

dff['Ticker'] = dff['Name'].apply(lambda x: getCompany(x)["symbol"])

In [5]:
dff

,Unnamed: 0,Name,val,Article Title,Ticker
0,0,Netflix,0.172789,"[""Meditation at dawn, miso soup for lunch, fit...",NFLX
1,1,Disney,0.150570,['Florida police chief is suspended for \'sayi...,DIS
2,2,Deere,0.128416,"[""Trump urges Americans to 'celebrate Easter a...",DE
3,3,Abbott Laboratories,0.123087,"[""'I'm banging my head against a wall': New Ha...",ABT
4,4,Walmart,0.119916,['Demand for thermal cameras soars as corporat...,WMT
5,5,Tyson Foods,0.119916,['Demand for thermal cameras soars as corporat...,TSN
6,6,Amazon.com,0.119916,['Demand for thermal cameras soars as corporat...,AMZN
7,7,Intel,0.114728,"[""Russia develops world's DEADLIEST sniper rif...",AIO
8,8,Microsoft,0.111002,"[""'Governments were NOT ready for coronavirus'...",MSFT
9,9,Dana,0.109763,"[""Amy Klobuchar 'has the best odds' of being n...",DAC


In [6]:
final = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends','Stock Splits', 'Name'])


# Extract historical stock data using yfinance

In [7]:
import yfinance as yf

for i in range(0,len(dff)):#define the ticker symbol
    tickerSymbol = dff["Ticker"][i]

#get data on this ticker
    tickerData = yf.Ticker(tickerSymbol)

#get the historical prices for this ticker
    tickerDf = tickerData.history(period='1y', start='2019-01-01', end='2020-01-01')

#see your data
    tickerDf["Name"] = dff["Name"][i]
    tickerDf.reset_index(level=0,inplace=True)
    final = final.append(tickerDf, ignore_index = True) 
    final.to_csv("tableau.csv", sep=',', index=False)
